In [40]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from random import randint
import numpy as np
import pandas as pd 
import torch.optim as optim
import os
from sklearn.preprocessing import LabelEncoder
from torch.utils.data import Dataset
import torch
from scipy.stats import vonmises
import numpy.fft as fft 
res = 8

In [60]:
##Define the network to train on the data 
class CircleNet(nn.Sequential): 
    def __init__(self, res, dropout, n_out): 
        super(CircleNet, self).__init__()
        self.n_in = res
        self.affine1 = nn.Linear(self.n_in, 2)
        self.drop1 = nn.Dropout(dropout)
        ##self.affine2 = nn.Linear(self.n_in//2, self.n_in//2)
        ##self.drop2 = nn.Dropout(dropout)
        self.affine3 = nn.Linear(2, n_out)
    def forward(self, x): 
        x = F.relu(self.affine1(x))
        x = self.drop1(x)
        ##x = F.relu(self.affine2(x))
        ##x = self.drop2(x)
        x = F.relu(self.affine3(x))
        x = F.softmax(x, dim = 1)
        return x

This is the Data Generation Code


In [83]:
## make some uniform circular data
res = 8
num_points = 100
num_samples = 100
data = np.zeros([num_points,res + 1])
for i in range(num_points): 
    for j in range(num_samples): 
        num = randint(0,res-1)
        data[i][num]+= 1
        data[i][res] = -1
np.savetxt("uniform8ptdata100test.csv", data, delimiter = ",")

In [ ]:
data = np.zeros([100,res+1])
for i in range(100):
    SAMPLE_SIZE = 100
    kappa = 1
    vm_dist = vonmises.rvs(kappa, size=SAMPLE_SIZE)
    vm_dist = (vm_dist + np.pi)/(2*np.pi)*(res*10)//10
    for k in range(SAMPLE_SIZE):
        data[i][int(vm_dist[k])] += 1
    rotate_amt = randint(0,res - 1)
    data[i,0:-1] = rotate(data[i,0:-1], rotate_amt)
    data[i,-1] = -3
np.savetxt("vonmises8ptdata100test.csv", data, delimiter = ",")    

In [62]:
##This is a class that allows us to load data from csv files and make them into datasets that the dataloader can use
class CircularDataset(Dataset): 
    def __init__(self, data_root): 
        self.data = []
        self.dist_coder = LabelEncoder()
        self.dist_list = []
        for path in data_root: 
            array = pd.read_csv(path)
            
            self.dist_list += [array.values[0,-1]] ##get the distribution type for each file and add it to the list
            if len(self.data) == 0: 
                self.data = array.values
            else:
                print(self.data.shape)
                print(array.values.shape)
                self.data = np.concatenate((self.data,array.values), axis = 0)
        self.dist_coder.fit(self.dist_list)
    def __getitem__(self,idx): 
        return self.data[idx][0:-1], self.to_one_hot(self.dist_coder,([self.data[idx,-1]]))[0]
    def __len__(self): 
        return len(self.data)
    def to_one_hot(self, codec, values):
        value_idxs = codec.transform(values)
        return torch.eye(len(codec.classes_))[value_idxs]

This is the code where we train the network on the data

In [93]:
def init_weights(m):
    if type(m) == nn.Linear:
        torch.nn.init.xavier_uniform_(m.weight)
        m.bias.data.fill_(0.00)
rate = .005
batchsize = 10
dataset = CircularDataset(["uniform8ptdata100.csv", "vonmises8ptdata100.csv"])
testdata = CircularDataset(["uniform8ptdata100test.csv", "vonmises8ptdata100test.csv"])
iterator = torch.utils.data.DataLoader(dataset, batch_size = batchsize, shuffle = True)
testiterator = torch.utils.data.DataLoader(testdata, batch_size = batchsize, shuffle = True)
net = CircleNet(8,0,2)
optimizer = optim.Adam(net.parameters(),lr = rate)
running_loss = 0
epochs = 10
net.apply(init_weights)
net.train()
loss_func = nn.CrossEntropyLoss()
for e in range(epochs): 
    print(running_loss)
    running_loss = 0
    accuracy = 0
    i = 0
    for dist, labels in iterator: 
        
        
        
        optimizer.zero_grad()
        out = net(dist.float()/20)
        
        loss = loss_func(out,torch.max(labels, 1)[1])
       
           
        loss.backward()
        optimizer.step()
        running_loss += loss
    for dist, labels in testiterator: 
        ps = net(dist.float()/20)
        
        i += 1
        top_p, top_class = ps.topk(1, dim=1)
       
        top_p2, top_class2 = labels.topk(1, dim = 1)
        
        equals = top_class == top_class2
        
        accuracy += torch.sum(equals.float())
        
    accuracy = accuracy/i/batchsize
    print(accuracy)
    accuracy = 0
    

(9999, 9)
(9999, 9)
(99, 9)
(99, 9)
0
tensor(0.8600)
tensor(1125.1398, grad_fn=<AddBackward0>)
tensor(0.8850)
tensor(972.6840, grad_fn=<AddBackward0>)
tensor(0.9000)
tensor(953.9169, grad_fn=<AddBackward0>)
tensor(0.9100)
tensor(947.4997, grad_fn=<AddBackward0>)
tensor(0.9150)
tensor(944.6589, grad_fn=<AddBackward0>)
tensor(0.9150)
tensor(942.9575, grad_fn=<AddBackward0>)
tensor(0.9250)
tensor(941.4842, grad_fn=<AddBackward0>)
tensor(0.9150)
tensor(940.9153, grad_fn=<AddBackward0>)
tensor(0.9200)
tensor(940.6455, grad_fn=<AddBackward0>)
tensor(0.9000)


In [101]:
def rotate(vec, num_rotations): 
    vec2 = np.zeros(len(vec))
    for i in range(len(vec)): 
        vec2[(i + num_rotations )% len(vec)] = vec[i]
    return vec2
data = np.zeros([100,res+1])
for i in range(100):
    SAMPLE_SIZE = 100
    kappa = 1
    vm_dist = vonmises.rvs(kappa, size=SAMPLE_SIZE)
    vm_dist = (vm_dist + np.pi)/(2*np.pi)*(res*10)//10
    
    for k in range(SAMPLE_SIZE):
        data[i][int(vm_dist[k])] += 1
    
    rotate_amt = randint(0,res - 1)
    data[i,0:-1] = rotate(data[i,0:-1], rotate_amt)
    data[i,-1] = -3
np.savetxt("vonmises8ptdata100test.csv", data, delimiter = ",")    

In [97]:
paramlist2 = []
for parameter in savednet2.parameters(): 
    paramlist2 += [parameter]

In [94]:
savednet2 = net

In [98]:
array2 = paramlist2[0].detach().numpy()
array2

array([[ 4.4111776 ,  1.8812704 , -1.0365207 , -1.0205503 , -0.12043339,
        -1.0698339 , -0.76035386,  1.8526413 ],
       [-1.8916395 , -2.3316643 , -1.0254574 ,  3.708905  ,  6.4014244 ,
         3.3557894 , -1.0132087 , -2.5337431 ]], dtype=float32)

In [99]:
np.matmul(array2,fft.ifft(np.identity(8)))

array([[ 0.51717466+0.j        ,  1.08125127-0.02763428j,
         0.76095235-0.00258182j,  0.05165149+0.04140743j,
         0.10629275+0.j        ,  0.05165149-0.04140743j,
         0.76095235+0.00258182j,  1.08125127+0.02763428j],
       [ 0.58380072+0.j        , -2.09111498+0.04754163j,
         0.81855638-0.01887959j,  0.01784901+0.05060379j,
         0.03397898+0.j        ,  0.01784901-0.05060379j,
         0.81855638+0.01887959j, -2.09111498-0.04754163j]])

In [100]:
fft.rfft(dataset.__getitem__(1)[0])

array([100.+0.j        ,  -1.+5.82842712j,  -8.-6.j        ,
        -1.-0.17157288j,   8.+0.j        ])

In [78]:
dataset = CircularDataset(["vonmises4.csv"])